In [ ]:
# read in patient key
root = UF.DataRoot(2)
LimbusKey = pd.read_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\LimbusKey.csv")
AllKey = pd.read_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\AllPatientKey_s.csv")

# Loop through treatments
treatments = AllKey["Treatment"].unique()
new_key = pd.DataFrame()
for t in treatments:
    AllKey_t = AllKey[AllKey["Treatment"] == t]
    LimbusKey_t = LimbusKey[LimbusKey["Treatment"] == t]

    # Loop through patients
    patIDs = LimbusKey_t["PatID"].unique()

    for pat in patIDs:
        pat_L = LimbusKey_t[LimbusKey_t["PatID"] == pat]
        pat_A = AllKey_t[AllKey_t["PatID"] == pat]
        print(pat_A)
        print(pat_L)
        # add date and days column from all to limbus key based on Scan
        for i in range(len(pat_L)):
            scan = pat_L.iloc[i]["Scan"]
            fraction = pat_A.iloc[i]["Fraction"]
            date = pat_A[pat_A["Scan"] == scan]["Date"].values[0]
            days = pat_A[pat_A["Scan"] == scan]["Days"].values[0]
            pat_L.at[pat_L.index[i], "Date"] = str(date)
            pat_L.at[pat_L.index[i], "Days"] = int(days)
            pat_L.at[pat_L.index[i], "Fraction"] = int(fraction)

        pat_L["Days"] = pat_L["Days"].astype(int)
        # sort by days
        pat_L = pat_L.sort_values(by=["Fraction"])

    # add to new key
        new_key = new_key.append(pat_L)

new_key.to_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\LimbusKey_s.csv", index=False)


        
        



In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

csvs = os.listdir("E:\\Aaron\ProstateMRL\Data\Paper1\HM-FSTP\\Longitudinal\ClusterLabels\\")
#csvs = [csv for csv in csvs if "HM" in csv]

fts_s = pd.read_csv("E:\\Aaron\ProstateMRL\Data\Paper1\\HM-FSTP\\Features\\SelectedFeatures_Longitudinal.csv")
fts_s = fts_s["Feature"].values
for csv in csvs:
    df = pd.read_csv("E:\\Aaron\ProstateMRL\Data\Paper1\\HM-FSTP\\Longitudinal\ClusterLabels\\" + csv)
    pat = str(csv)[:-4]
    df["Selected"] = df["Feature"].apply(lambda x: x in fts_s)
    # print where selected is True
    df = df[['Feature', 'Cluster', 'Fraction', 'FeatureChange', 'Selected']]
    df['Feature'] = df['Feature'].str.replace('original_', '')
    clusters = df["Cluster"].unique()
    clusters = sorted(clusters, key=lambda x: int(x))
    for c in clusters:
        df_c = df[df["Cluster"] == c]
        df_c = df_c.sort_values(by = ["Fraction"])

        # get selected features
        selected_fts = df_c[df_c["Selected"] == True]["Feature"].unique()
        if len(selected_fts) == 0:
            sf_str = "No features selected"
        
        number_fts = "Total number of feature(s) in Cluster {}: {}\nNumber of selected features: {}\n".format(c, df_c["Feature"].nunique(), len(selected_fts) )
        text_str = selected_fts
        text_str = '\n'.join(text_str)
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.7)
        fts = df_c["Feature"].values
        plt.figure(figsize=(10, 10))
        plt.title(" Cluster " + str(c))
        sns.set_theme(style="whitegrid")
        sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
        for ft in fts:
            df_ft = df_c[df_c["Feature"] == ft]
            values = df_ft["FeatureChange"].values
            fractions = df_ft["Fraction"].values
            colour = "blue" if df_ft["Selected"].values[0] else "grey"
            l = df_ft["Feature"].values[0]
            plt.plot(fractions, values, label = l, color = colour)
            #plt.scatter(fractions, values, color = colour)
        plt.xlabel("Fraction", fontsize = 20)
        plt.ylabel("Feature Change", fontsize = 20)
        plt.xticks(np.arange(1, 5.1, 1))
        plt.xlim(1, 5)
        #plt.ylim(-1, 1)
        # add text box
        #plt.text(0.05, 0.95, (number_fts + text_str), transform=plt.gca().transAxes, fontsize=14, verticalalignment='top', bbox=props)
        
        #plt.legend(title = "Feature Selected", bbox_to_anchor=(1, 0.6), labels = ["Yes", "No"])
        plt.title("Cluster " + str(c), fontsize = 30)
        plt.savefig("E:\\Aaron\ProstateMRL\Data\Paper1\HM-FSTP\Longitudinal\ClusterPlots\\Run2_" + str(pat) + "_Cluster" + str(c) + ".png", bbox_inches = "tight")
        #plt.show()
        plt.close()






In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

dir = "E:\\Aaron\\ProstateMRL\\Data\\Paper1\\NormSummary\\"

files = os.listdir(dir)

result = pd.DataFrame()

for file in files:
    # loop through txt files and get lines that contain Before:
    with open(dir + file, "r") as f:
        lines = f.readlines()
        # filter so only lines with Before: are kept
        # get line with 
        lines = [line for line in lines if "Longitudinal" in line]

        r_lines = [line for line in lines if "Removed:" in line]
        s_line = [line for line in lines if "Selected:" in line]
        
        ICC = int(r_lines[0][-3:])
        Vol = int(r_lines[1][-3:])
        Sel = int(s_line[0][-3:])

        result = result.append({"Norm": file[:-4], "ICC": ICC, "Volume": Vol, "Selected": Sel}, ignore_index=True) 
result = result.melt(id_vars=["Norm"], var_name="Stage", value_name="Number")

# order result so plot is in correct order
result["Norm"] = pd.Categorical(result["Norm"], ["Raw", "HM-FS", "HM-TP", "HM-FSTP", "Med-Pros", "Med-Psoas"])
result["Stage"] = pd.Categorical(result["Stage"], ["ICC", "Volume", "Selected"])

# bar plot results
plt.figure(figsize=(10, 10))
sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.barplot(x="Norm", y="Number", data=result, hue="Stage")
plt.xlabel("Normalisation", fontsize = 20)
plt.ylabel("Number of Features", fontsize = 20)
plt.title("Number of Features Removed or Selected", fontsize = 26)



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


df = pd.read_csv("E:\\Aaron\ProstateMRL\Code\PatKeys\\MedianSignalNorm.csv")
df = df.drop(columns=["Unnamed: 0"])

# pick a random patient
pat = df["PatID"].sample().values[0]

# change Mask variables to Prostate and Psoas
df["Mask"] = df["Mask"].apply(lambda x: "Prostate" if x == "shrunk_pros" else "Psoas")
df = df[df["PatID"] == pat]
# plot signal over time for each normalisation in facet grid
g = sns.FacetGrid(df, col="Norm", hue="Mask", col_wrap=3, height=5, aspect=1.5)
g.map(sns.lineplot, "Scan", "Median", marker="o")
g.set_axis_labels("Fraction", "Median Signal")
g.set_titles("{col_name}")
g.add_legend()
plt.suptitle("Median Signal for Patient " + str(pat), fontsize = 30)
plt.subplots_adjust(top=0.9)
plt.show()



In [ ]:
import pandas as pd
from sklearn import preprocessing
from tqdm import tqdm
root = "E:\\"
Norm = "HM-FS"

#df_all = pd.read_csv(root + "Aaron\ProstateMRL\Data\Paper1\\" + Norm + "\\Features\\Longitudinal_All_fts_Baseline.csv")
# remove minimum feature
df_all = pd.read_csv(root + "Aaron\ProstateMRL\Data\Paper1\\" + Norm + "\\Features\\Longitudinal_Limbus_fts_Baseline.csv")
df_all = df_all[df_all["Feature"] != "original_firstorder_Minimum"]


df_res = pd.DataFrame()
for ft in tqdm(df_all["Feature"].unique()):
    df_ft = df_all[df_all["Feature"] == ft]
    ft_vals = df_ft["FeatureValue"].values
    ft_change = df_ft["FeatureChange"].values
    
    ft_vals01 = preprocessing.minmax_scale(ft_vals, feature_range=(0, 1))
    ft_change01 = preprocessing.minmax_scale(ft_change, feature_range=(0, 1))
    
    df_ft.loc[df_all["Feature"] == ft, "FeatureValue"] = ft_vals01
    df_ft.loc[df_all["Feature"] == ft, "FeatureChange"] = ft_change01
    # append 
    df_res = df_res.append(df_ft, ignore_index=True)

df_res.to_csv(root + "Aaron\ProstateMRL\Data\Paper1\\" + Norm + "\\Features\\Longitudinal_Limbus_fts_BaselineRS.csv")

    

# pivot table so feature values and changes are in columns
# plot features over time for each patient in facet grid


    

In [ ]:
# pivot df long 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


df_l = df_all.melt(id_vars=["PatID", "Scan", "Mask", "Fraction", "Days", "Feature"], var_name="FeatureType", value_name="Value")

df_l = df_l[df_l["FeatureType"].isin(["FeatureRS11", "FeatureRS01", "FeatureChangeRS11", "FeatureChangeRS01"])]
# add small random noise to fraction to avoid overlapping points
df_l["Fraction"] = df_l["Fraction"] + np.random.normal(0, 0.05, df_l.shape[0])

# plot features over time for each patient in facet grid
for ft in df_l["Feature"].unique()[0:10]:
    df_l_ft = df_l[df_l["Feature"] == ft]
    g = sns.FacetGrid(df_l_ft, col="PatID", hue="FeatureType", col_wrap=5, height=5, aspect=1.5)
    g.map(sns.lineplot, "Fraction", "Value", marker="o")
    g.set_axis_labels("Fraction", "Feature Value")
    g.set_titles("{col_name}")
    g.add_legend()
    plt.suptitle(ft, fontsize = 30)
    plt.subplots_adjust(top=0.9)
    plt.show()
    plt.close()



In [18]:
import pandas as pd


def ClusterCount(root, Norm, output, tag):
    '''
    Summarises clustering results
    '''
    patIDs = ['653', '713', '752', '826', '1088', '1089', '1118', '1303', '1307', '1464', '1029',
    '1302', '1431', '1481', '1540', '1553', '1601', '1642', '829', '955']
    dir = root + "\\Aaron\\ProstateMRL\\Data\\Paper1\\" + Norm + "\\Longitudinal\\ClusterLabels\\"
    #patIDs = UF.SABRPats()
    df_result = pd.DataFrame()

    for pat in patIDs:

        df = pd.read_csv(root + "\\Aaron\\ProstateMRL\\Data\\Paper1\\" + Norm + "\\Longitudinal\\ClusterLabels\\" + pat + "_" + tag + ".csv")
        df = df[["Feature", "Cluster"]]
        df = df.drop_duplicates()
        # sort by cluster
        df = df.sort_values(by=["Cluster"])
        # turn value counts into a dataframe
        df = df["Cluster"].value_counts().rename_axis("Cluster").reset_index(name="Counts")
        # set PatID as index
        df["PatID"] = pat
        # set PatID as index
        df.set_index("PatID", inplace=True)
            
        # append to result
        df_result = df_result.append(df, ignore_index=False)
    
    
    
    # get number of clusters per patient
    df_numclust= df_result.groupby("PatID")["Cluster"].count()
    
    # get number of clusters with more than 3 features
    df_stable = df_result[df_result["Counts"] > 3]
    df_stable = df_stable.groupby("PatID")["Cluster"].count()
    
    # merge number of clusters with more than 3 features and number of clusters
    df_numclust = df_numclust.rename_axis("PatID").reset_index(name="NumClusters")
    df_stable = df_stable.rename_axis("PatID").reset_index(name="NumStableClusters")
    df_numclust = pd.merge(df_numclust, df_stable, on="PatID")
    # get mean number of clusters
    meanclust = df_numclust["NumClusters"].mean()
    # get mean number of stable clusters
    meanstable = df_numclust["NumStableClusters"].mean()
    # get mean number of features per cluster
    df_numfts = df_result.groupby("PatID")["Counts"].mean()
    df_numfts = df_numfts.rename_axis("PatID").reset_index(name="MeanFeaturesperCluster")
    df_numclust = pd.merge(df_numclust, df_numfts, on="PatID")
    # get mean number of features per stable cluster
    df_numfts = df_result[df_result["Counts"] > 3]
    df_numfts = df_numfts.groupby("PatID")["Counts"].mean()
    df_numfts = df_numfts.rename_axis("PatID").reset_index(name="MeanFeaturesperStableCluster")
    df_numclust = pd.merge(df_numclust, df_numfts, on="PatID")

    
    print("Mean number of clusters: " + str(meanclust))
    print("Mean number of stable clusters: " + str(meanstable))
    print("Mean number of features per cluster: " + str(df_numclust["MeanFeaturesperCluster"].mean()))
    print("Mean number of features per stable cluster: " + str(df_numclust["MeanFeaturesperStableCluster"].mean()))


In [19]:
root = "E:\\"
Norm = "HM-FS"
output = False
tag = "Baseline"
ClusterCount(root, Norm, output, tag)

Mean number of clusters: 12.85
Mean number of stable clusters: 7.3
Mean number of features per cluster: 4.717774541634837
Mean number of features per stable cluster: 6.828075396825396


In [20]:
root = "E:\\"
Norm = "HM-FS"
output = False
tag = "BaselineRS"
ClusterCount(root, Norm, output, tag)

Mean number of clusters: 10.05
Mean number of stable clusters: 7.6
Mean number of features per cluster: 5.91787878787879
Mean number of features per stable cluster: 7.10952380952381


In [21]:
root = "E:\\"
Norm = "HM-FS"
output = False
tag = "t_val_15"
ClusterCount(root, Norm, output, tag)

Mean number of clusters: 15.25
Mean number of stable clusters: 6.6
Mean number of features per cluster: 4.013792871645038
Mean number of features per stable cluster: 6.717460317460318


In [29]:
import pandas as pd

df = pd.read_csv("E:\\Aaron\\ProstateMRL\\Data\\Paper1\\HM-FS\\Features\\Longitudinal_All_fts_Baseline.csv")
ok_pats = df.loc[df["Fraction"] == 5]["PatID"].unique()
# need_pats = df.loc[df["Fraction"] == 0]["PatID"].unique()
print(ok_pats)
need_pats = []
#need_pats = [x for x in need_pats if x not in ok_pats]
#print(need_pats)

for pat in df["PatID"].unique():
    tps = len(df.loc[df["PatID"] == pat]["Fraction"].unique())
    Scans = df.loc[df["PatID"] == pat]["Scan"].unique()
    if tps < 5:
        print("Pat: {} Timepoints: {}, Scans: {}".format(pat, tps, Scans))
        need_pats.append(pat)
print(need_pats)


[ 653  713  826 1029 1302 1431 1481 1540 1553 1601 1642  829]
Pat: 752 Timepoints: 4, Scans: ['MR5' 'MR9' 'MR12' 'MR14']
Pat: 1088 Timepoints: 4, Scans: ['MR10' 'MR13' 'MR19' 'MR25']
Pat: 1089 Timepoints: 4, Scans: ['MR6' 'MR12' 'MR15' 'MR20']
Pat: 1118 Timepoints: 4, Scans: ['MR5' 'MR10' 'MR15' 'MR20']
Pat: 1303 Timepoints: 4, Scans: ['MR5' 'MR8' 'MR13' 'MR19']
Pat: 1307 Timepoints: 4, Scans: ['MR6' 'MR11' 'MR14' 'MR18']
Pat: 1464 Timepoints: 4, Scans: ['MR1' 'MR3' 'MR4' 'MR5']
Pat: 955 Timepoints: 4, Scans: ['MR1' 'MR6' 'MR16' 'MR22']
[752, 1088, 1089, 1118, 1303, 1307, 1464, 955]
